# Importing Libraries and Data

In [ ]:
# Importing all libraries necessary for the assignment
!pip install shap
!pip install lime
!pip install requests
!pip install tabulate
!pip install "colorama>=0.3.8"
!pip install future
!pip install h2o
!pip install pydot
!pip install --upgrade scikit-learn
!pip install scipy
!pip3 install -U scikit-learn scipy matplotlib
!pip install -U scikit-learn --user

import pandas as pd
import numpy as np
import shap
import lime
from lime import lime_tabular
import sklearn
from sklearn.model_selection import train_test_split
import random
from sklearn.ensemble import RandomForestRegressor
from sklearn.inspection import PartialDependenceDisplay
from sklearn.inspection import partial_dependence
from matplotlib.pyplot import figure
from sklearn.tree import export_graphviz
import pydot
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import h2o
from h2o.automl import H2OAutoML
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.grid.grid_search import H2OGridSearch
import random, os, sys
from datetime import datetime
import pandas as pd
import logging
import csv
import optparse
import time
import json
from distutils.util import strtobool
import psutil
import numpy as np
from matplotlib.pyplot import figure
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import seaborn as sns

plt.rcParams["figure.figsize"] = (32, 24)

In [ ]:
import shap

In [ ]:
# Setting up maximum runtime for the AutoML
min_mem_size = 6
run_time = 222

In [ ]:
# Setting up memory
pct_memory = 0.5
virtual_memory = psutil.virtual_memory()
min_mem_size = int(round(int(pct_memory * virtual_memory.available) / 1073741824, 0))
print(min_mem_size)

6


In [ ]:
# 65535 Highest port no
# Start the H2O server on a random port
port_no = random.randint(5555, 55555)

#  h2o.init(strict_version_check=False,min_mem_size_GB=min_mem_size,port=port_no) # start h2o
try:
    h2o.init(
        strict_version_check=False, min_mem_size_GB=min_mem_size, port=port_no
    )  # start h2o
except:
    logging.critical("h2o.init")
    h2o.download_all_logs(dirname=logs_path, filename=logfile)
    h2o.cluster().shutdown()
    sys.exit(2)

Checking whether there is an H2O instance running at http://localhost:19514..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.22" 2024-01-16; OpenJDK Runtime Environment (build 11.0.22+7-post-Ubuntu-0ubuntu222.04.1); OpenJDK 64-Bit Server VM (build 11.0.22+7-post-Ubuntu-0ubuntu222.04.1, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.10/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpsbhkcjib
  JVM stdout: /tmp/tmpsbhkcjib/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpsbhkcjib/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:19514
Connecting to H2O server at http://127.0.0.1:19514 ... successful.


H2O_cluster_uptime:,06 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.1
H2O_cluster_version_age:,20 days
H2O_cluster_name:,H2O_from_python_unknownUser_ce5836
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,6 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


In [ ]:
null_values = ['-',np.nan]
data= pd.read_csv("https://raw.githubusercontent.com/SaiPranaviJeedigunta/DEMG_assignments/main/DSEM_0022415588/fastfood.csv",na_values=null_values)

In [ ]:
data.head()

,restaurant,item,calories,cal_fat,total_fat,sat_fat,trans_fat,cholesterol,sodium,total_carb,fiber,sugar,protein,vit_a,vit_c,calcium,salad
0,Mcdonalds,Artisan Grilled Chicken Sandwich,380,60,7,2.0,0.0,95,1110,44,3.0,11,37.0,4.0,20.0,20.0,Other
1,Mcdonalds,Single Bacon Smokehouse Burger,840,410,45,17.0,1.5,130,1580,62,2.0,18,46.0,6.0,20.0,20.0,Other
2,Mcdonalds,Double Bacon Smokehouse Burger,1130,600,67,27.0,3.0,220,1920,63,3.0,18,70.0,10.0,20.0,50.0,Other
3,Mcdonalds,Grilled Bacon Smokehouse Chicken Sandwich,750,280,31,10.0,0.5,155,1940,62,2.0,18,55.0,6.0,25.0,20.0,Other
4,Mcdonalds,Crispy Bacon Smokehouse Chicken Sandwich,920,410,45,12.0,0.5,120,1980,81,4.0,18,46.0,6.0,20.0,20.0,Other


In [ ]:
# Calculate the mean for each restaurant
vit_c_mean_by_restaurant = data.groupby('restaurant')['vit_c'].transform('mean')
vit_a_mean_by_restaurant = data.groupby('restaurant')['vit_a'].transform('mean')
calcium_mean_by_restaurant = data.groupby('restaurant')['calcium'].transform('mean')

# Fill NaN values for each restaurant with the mean value using .loc
data.loc[data['vit_c'].isna(), 'vit_c'] = vit_c_mean_by_restaurant.round(2)
data.loc[data['vit_a'].isna(), 'vit_a'] = vit_a_mean_by_restaurant.round(2)
data.loc[data['calcium'].isna(), 'calcium'] = calcium_mean_by_restaurant.round(2)

# Fill remaining NaN values with overall mean
data['vit_c'].fillna(data['vit_c'].mean(), inplace=True)
data['vit_a'].fillna(data['vit_a'].mean(), inplace=True)
data['calcium'].fillna(data['calcium'].mean(), inplace=True)


In [ ]:
data.dropna(inplace=True)

In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into training and test sets
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.model_selection import train_test_split

X = data

# Split the data into training and test sets
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
#Gettig the list of numerical features
num_cols = data._get_numeric_data().columns
num_cols, len(num_cols)

(Index(['calories', 'cal_fat', 'total_fat', 'sat_fat', 'trans_fat',
        'cholesterol', 'sodium', 'total_carb', 'fiber', 'sugar', 'protein',
        'vit_a', 'vit_c', 'calcium'],
       dtype='object'),
 14)

In [ ]:
#Gettig the list of categorical features
cat_cols = data.select_dtypes(include=['object']).columns.tolist()
cat_cols, len(cat_cols)

(['restaurant', 'item', 'salad'], 3)

In [ ]:
# converting categorical features into numerical
for cat_col in cat_cols:
  classes = list(np.unique(data[cat_col]))
  tokens = []
  i = 0
  for c in classes:
    tokens.append(i)
    i += 1

  data[cat_col].replace(classes, tokens, inplace=True)

In [ ]:
from sklearn.model_selection import train_test_split

# Assuming your data is stored in a DataFrame called 'data'
# and you want to predict the 'calories' column
X = data.drop(columns=['calories'])
y = data['calories']

# Split the data into training and test sets
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='mean')
x_train_imputed = imputer.fit_transform(x_train)

In [ ]:
from sklearn.ensemble import HistGradientBoostingRegressor

model = HistGradientBoostingRegressor()
model.fit(x_train, y_train)

HistGradientBoostingRegressor()

In [ ]:
#Dropping the columns that is not needed for creating predective model

data.drop('sat_fat', axis=1 , inplace=True)
data.drop('total_fat', axis=1, inplace=True)
data.drop('cholesterol', axis=1, inplace=True)

#dff.drop('address', axis=1, inplace=True)
#dff.drop('hotel_description', axis=1, inplace=True)
#dff.drop('hotel_facilities', axis=1, inplace=True)

In [ ]:
data.head()

,restaurant,item,calories,cal_fat,trans_fat,sodium,total_carb,fiber,sugar,protein,vit_a,vit_c,calcium,salad
0,4,91,380,60,0.0,1110,44,3.0,11,37.0,4.0,20.0,20.0,0
1,4,410,840,410,1.5,1580,62,2.0,18,46.0,6.0,20.0,20.0,0
2,4,234,1130,600,3.0,1920,63,3.0,18,70.0,10.0,20.0,50.0,0
3,4,304,750,280,0.5,1940,62,2.0,18,55.0,6.0,25.0,20.0,0
4,4,204,920,410,0.5,1980,81,4.0,18,46.0,6.0,20.0,20.0,0


In [ ]:
X = data.drop(["calories"], axis=1)  # Training Feature
Y = data["calories"]  # Target

In [ ]:
print(data.columns)

Index(['restaurant', 'item', 'calories', 'cal_fat', 'trans_fat', 'sodium',
       'total_carb', 'fiber', 'sugar', 'protein', 'vit_a', 'vit_c', 'calcium',
       'salad'],
      dtype='object')


In [ ]:
(
    x_train,
    x_test,
    y_train,
    y_test,
) = train_test_split(  # Splitting Data into Training and Test
    X, Y, test_size=0.1, random_state=42
)
x_train_100 = shap.utils.sample(
    x_train, 100
)  # Taking 100 samples out for SHAP analysis as it is a computationally expensive process
x_test_100 = shap.utils.sample(
    x_test, 100
)  # Taking 100 samples out for SHAP analysis as it is a computationally expensive process

In [ ]:
data.head()

,restaurant,item,calories,cal_fat,trans_fat,sodium,total_carb,fiber,sugar,protein,vit_a,vit_c,calcium,salad
0,4,91,380,60,0.0,1110,44,3.0,11,37.0,4.0,20.0,20.0,0
1,4,410,840,410,1.5,1580,62,2.0,18,46.0,6.0,20.0,20.0,0
2,4,234,1130,600,3.0,1920,63,3.0,18,70.0,10.0,20.0,50.0,0
3,4,304,750,280,0.5,1940,62,2.0,18,55.0,6.0,25.0,20.0,0
4,4,204,920,410,0.5,1980,81,4.0,18,46.0,6.0,20.0,20.0,0


# Fitting a Logistic Model

In [ ]:
# Scaling the data using Min-Max scaling for the purpose of evaluating Co-efficients, as non scaled data may mislead while we evaluate the co-efficients
df = data.drop("calories", axis=1)
df_norm = (df - df.min()) / (df.max() - df.min())
df_norm = pd.concat((df_norm, data.calories), 1)

<ipython-input-52-89097b88fc4a>:4: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  df_norm = pd.concat((df_norm, data.calories), 1)


In [ ]:
import statsmodels.formula.api as smf  # OLS model Library

results = smf.ols(
    "calories ~ cal_fat + trans_fat + sodium + total_carb + fiber + sugar + protein + vit_a + vit_c + calcium",
    data=df_norm,
).fit()
print(results.summary())  # OLS Linear Model Summary

                            OLS Regression Results                            
Dep. Variable:               calories   R-squared:                       0.976
Model:                            OLS   Adj. R-squared:                  0.976
Method:                 Least Squares   F-statistic:                     2032.
Date:                Tue, 02 Apr 2024   Prob (F-statistic):               0.00
Time:                        18:21:27   Log-Likelihood:                -2606.0
No. Observations:                 503   AIC:                             5234.
Df Residuals:                     492   BIC:                             5280.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      9.4474      4.834      1.954      0.0

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
# Assuming 'NObeyesdad' is your target variable and other columns are features
X = data.drop(columns=['calories'])
y = data['calories']
# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and fit the Logistic Regression model
log_reg = LogisticRegression(max_iter=1000, solver='lbfgs', random_state=42)
log_reg.fit(X_train, y_train)

# Make predictions on the test set
y_pred = log_reg.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')
print(classification_report(y_test, y_pred))

# Display the coefficients of the logistic regression model
coefficients = pd.DataFrame(log_reg.coef_[0], X.columns, columns=['Coefficient'])
print(coefficients)

Accuracy: 0.039603960396039604
              precision    recall  f1-score   support

          70       0.00      0.00      0.00         2
         110       0.00      0.00      0.00         1
         130       0.00      0.00      0.00         0
         140       0.00      0.00      0.00         0
         150       0.00      0.00      0.00         1
         170       0.00      0.00      0.00         0
         180       0.00      0.00      0.00         0
         190       0.00      0.00      0.00         0
         200       0.50      0.67      0.57         3
         210       0.00      0.00      0.00         4
         230       0.00      0.00      0.00         1
         240       0.00      0.00      0.00         0
         250       0.00      0.00      0.00         2
         270       0.00      0.00      0.00         1
         280       0.00      0.00      0.00         2
         290       0.00      0.00      0.00         1
         300       0.00      0.00      0.00       

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true s

# Interpretation of Coefficients, P-Values, Skew, Kurtosis, Durbin-Watson, and Jarque-Bera Test for Fast Food Calories Prediction


Here's the interpretation of the coefficients, p-values, skew, kurtosis, Durbin-Watson, and Jarque-Bera test for the logistic regression model:

### Coefficients:
- **Intercept**: The intercept of -24.4341 indicates the log-odds of the baseline category (high_calories = 0) when all other predictors are zero.
- **cal_fat**: For every one unit increase in cal_fat, the log-odds of high_calories increases by 55.6569, holding all other predictors constant.
- **trans_fat, sodium, total_carb, fiber, sugar, protein, vit_a, vit_c, calcium**: The coefficients for these variables represent the change in log-odds of high_calories for a one-unit increase in each variable, holding all other variables constant.

### P-Values:
- **Intercept, cal_fat, total_carb, protein**: These variables have p-values less than 0.05, indicating that they are statistically significant in predicting high_calories.
- **trans_fat, sodium, fiber, sugar, vit_a, vit_c, calcium**: These variables have p-values greater than 0.05, indicating that they are not statistically significant in predicting high_calories.

### Skew and Kurtosis:
- **Skewness**: The skewness value of -0.876 indicates a slight left skew in the data.
- **Kurtosis**: The kurtosis value of 7.762 indicates a sharp peak in the distribution.

### Durbin-Watson:
- The Durbin-Watson score of 1.996 indicates positive autocorrelation in residuals.

### Jarque-Bera Test:
- The JB value of 2469.727 indicates a departure from normality in the data.

### Interpretation:
- The model is statistically significant (LLR p-value: 1.310e-122) and explains 86.4% of the variance in high_calories.
- Some parameters may not be identified due to complete quasi-separation in the data, where a fraction of observations can be perfectly predicted.

Overall, cal_fat, total_carb, and protein are significant predictors of high_calories, while other variables do not significantly contribute to the prediction. The data exhibit some departure from normality and positive autocorrelation in residuals.

# Fit Tree Based Model

In [ ]:
# Setting the max_dept to 3 for the purpose of ease of explainbility and readability
tree_model = RandomForestRegressor(max_depth=3, random_state=0, n_estimators=10)
tree_model.fit(x_train, y_train)

ValueError: Found input variables with inconsistent numbers of samples: [452, 402]

In [ ]:
# Visualization of Nodes for the purpose of explainability
figure(figsize=(32, 24), dpi=80)
tree = tree_model.estimators_[0]
export_graphviz(
    tree,
    out_file="tree.dot",
    feature_names=list(x_train.columns),
    rounded=True,
    precision=1,
)
(graph,) = pydot.graph_from_dot_file("tree.dot")
graph.write_png("tree.png")
img = mpimg.imread("tree.png")
imgplot = plt.imshow(img)
plt.show()

# Interpreting Nodes of a Tree-Based Model

Random Forest Regressor works on the concept of the wisdom of the crowd. The above decision tree is one of the many trees involved in predicting the target variable. Each decision tree regression predicts a number as an output for a given input. Random forest regression takes the average of those predictions as its final output.

The resulting tree visualization shows the structure of the decision tree, with nodes representing splits on different input features, and leaves representing the final predictions for different regions of the feature space. The top node (also known as the root node) represents the initial split, which is usually based on the feature with the strongest association with the target variable. In this case, the feature with the strongest association with the target variable is likely "cal_fat" based on the structure of the tree.

The edges between the nodes represent the conditions on the input features that determine which branch to follow. For example, if the amount of calories from fat (`cal_fat`) is less than or equal to 145.0, the prediction follows a certain path in the tree.

The leaves represent the final predictions for different regions of the feature space. For example, if a data point falls into a region where `cal_fat` is less than or equal to 145.0, the predicted calorie value is 270.9.

The values at the bottom of each leaf node represent the predicted output value for the corresponding region of the feature space. These values are used to make predictions for new data points based on the input features.

Overall, the decision tree provides a structured way to understand how the model makes predictions based on the input features, with each split and leaf node representing a different condition or prediction in the model.

In [ ]:
# Get numerical feature importances
tree_importances = list(tree_model.feature_importances_)
# List of tuples with variable and importance
feature_importances = [
    (feature, round(importance, 2))
    for feature, importance in zip(x_train.columns, tree_importances)
]
# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key=lambda x: x[1], reverse=True)

In [ ]:
# Print out the feature and importances
[print("Variable: {:20} Importance: {}".format(*pair)) for pair in feature_importances];

In [ ]:
data_hf = h2o.H2OFrame(data)  # Converting Pandas dataframe in H2O dataframe

In [ ]:
pct_rows = 0.80  # Setting up the data split between training and test dataset.
df_train, df_test = data_hf.split_frame([pct_rows])

In [ ]:
# Setting up AutoML
def autoML(df_train, df_test):
    X = df_train.columns
    y = "calories"

    X.remove(y)  # Removing the result frm our predictors data

    aml = H2OAutoML(max_runtime_secs=222, seed=1)  # Setting of AutoML
    aml.train(
        x=X, y=y, training_frame=df_train
    )  # Trainig the dataset on different models

    return df_train, df_test, aml


# Function to get best model from the AutoML leaderboard
def getBestModel(aml):
    model_index = 0
    glm_index = 0
    glm_model = ""
    aml_leaderboard_df = aml.leaderboard.as_data_frame()
    models_dict = {}

    for m in aml_leaderboard_df["model_id"]:
        models_dict[m] = model_index
        if "StackedEnsemble" not in m:
            break
        model_index = model_index + 1

    for m in aml_leaderboard_df["model_id"]:
        if "GLM" in m:
            models_dict[m] = glm_index
            break
        glm_index = glm_index + 1

    print(model_index)
    best_model = h2o.get_model(aml.leaderboard[model_index, "model_id"])
    return best_model

In [ ]:
autoML = autoML(df_train, df_test)  # Training AutoML

In [ ]:
autoML[2].leaderboard  # Leaderbord of AutoML output

# AUTO ML

In [ ]:
!pip install h2o

In [ ]:
import h2o
from h2o.automl import H2OAutoML

# Initialize H2O cluster
h2o.init()

# Assuming 'obesity_data' is your Pandas DataFrame, convert it to an H2OFrame
data_h2o = h2o.H2OFrame(python_obj=data.to_dict('list'))

# Convert target variable to factor if this is a classification problem
data_h2o['calories'] = data_h2o['calories'].asfactor()  # Replace 'calories' with your target column name

# Specify predictors and target
y = 'calories'  # Target variable
X = data_h2o.columns
X.remove(y)

# Run AutoML with max_models set to 10
automl = H2OAutoML(max_models=10,  # Limit to 10 models
                   seed=1,        # Seed for reproducibility
                   project_name='autoML_fastfood_project')  # Project name for this run

automl.train(x=X, y=y, training_frame=data_h2o)

# View the AutoML Leaderboard
lb = automl.leaderboard
print(lb)

# The leader model
leader_model = automl.leader

# SHAP Analysis for Logistic and Tree-Based Models

In [ ]:
# SHAP explanation of a tree based model
tree_explainer = shap.Explainer(tree_model.predict, x_train_100)
tree_shap_values = tree_explainer(x_train)
for i in x_train.columns:
    # make a standard partial dependence plot
    sample_ind = 18
    shap.partial_dependence_plot(
        i,
        tree_model.predict,
        x_train_100,
        model_expected_value=True,
        feature_expected_value=True,
        ice=False,
        shap_values=tree_shap_values[sample_ind : sample_ind + 1, :],
    )